# Description 

It's Thanksgiving eve and you're expecting guests over for dinner tomorrow. Unfortunately, you were browsing memes all day and cannot go outside to buy the ingredients needed to make your famous pies. You find some spare ingredients, and make do with what you have. You know only two pie recipes, and they are as follows:

#### Pumpkin Pie
* **1** scoop of synthetic pumpkin flavouring (Hey you're a programmer not a cook) 
* **3** eggs 
* **4** cups of milk 
* **3** cups of sugar 

#### Apple Pie
* **1** apple
* **4** eggs
* **3** cups of milk
* **2** cups of sugar 

Your guests have no preference of one pie over another, and you want to make the maximum number of (any kind) of pies possible with what you have. You cannot bake fractions of a pie, and cannot use fractions of an ingredient (So no 1/2 cup of sugar or anything like that) 

# Input Format

You will be given a string of 4 numbers separated by a comma, such as `10,14,10,42,24`. Each number is a non-negative integer. The numbers represent the number of synthetic pumpkin flavouring, apples, eggs, milk and sugar you have (In the units represented in the recipes). 

For instance, in the example input `10,14,10,42,24`, it would mean that you have

* **10** scoops of synthetic pumpkin flavouring
* **14** apples
* **10** eggs
* **42** cups of milk
* **24** cups of sugar 

# Output Format

Display the number of each type of pie you will need to bake. For the example input, an output would be

    3 pumpkin pies and 0 apple pies

# Challenge Inputs

    10,14,10,42,24
    12,4,40,30,40
    12,14,20,42,24
    

# Challenge Outputs

    3 pumpkin pies and 0 apple pies
    5 pumpkin pies and 3 apple pies
    5 pumpkin pies and 1 apple pies

# Hint
Look into [linear programming](https://www.purplemath.com/modules/linprog.htm)

# Credit

This challenge was suggested by user /u/Gavin_Song, many thanks! If you have an idea for a challenge please share it on /r/dailyprogrammer_ideas and there's a good chance we'll use it. 

In [82]:
from pulp import *
from collections import OrderedDict
from operator import mul

In [116]:
RECIPES_1 = OrderedDict({"pumpkin": [1, 0, 3, 4, 3], 
             "apple": [0, 1, 4, 3, 2]})
INGREDIENTS_1 = [10, 14, 10, 42, 24]

In [117]:
INGREDIENTS_2 = [12, 14, 20, 42, 24]

In [148]:
INGREDIENTS_3 = [12, 4, 40, 30, 40]

In [109]:
def print_results(prob):
    print("Status:", LpStatus[prob.status])

    for v in prob.variables():
        print(v.name, "=", v.varValue)

    # The optimised objective function value is printed to the screen
    print("total pies produced = ", value(prob.objective))

In [110]:
def solve(recipes, total_ingredients):
    prob = LpProblem("pies", LpMaximize)

    # variables
    variables = OrderedDict({pie:LpVariable(pie, lowBound=0, cat='Integer') for pie in recipes.keys()})
    
    # objective
    prob += sum(variables.values())
    
    # constraints
    ingredients_by_type = zip(*recipes.values())
    
    for recipe_parts, limit in zip(ingredients_by_type, total_ingredients):
        prob += sum([mul(var, num) for num, var in zip(recipe_parts, variables.values())]) <= limit
        
    GLPK().solve(prob)
    
    return prob

In [145]:
print_results(solve(RECIPES_1, INGREDIENTS_1))

Status: Optimal
apple = 0
pumpkin = 3
total pies produced =  3


In [146]:
print_results(solve(RECIPES_1, INGREDIENTS_2))

Status: Optimal
apple = 1
pumpkin = 5
total pies produced =  6


In [149]:
print_results(solve(RECIPES_1, INGREDIENTS_3))

Status: Optimal
apple = 3
pumpkin = 5
total pies produced =  8
